In [6]:
pwd

'/Users/johnnytay/Library/CloudStorage/OneDrive-Personal/My NUS Mtech EBAC course/Semester 3/Practice Module/bead_pyspark/notebooks'

In [46]:
from datetime import datetime
import pandas as pd
import praw
import json
import pyarrow as pa
import pyarrow.parquet as pq

from dotenv import load_dotenv
import os

# Load environment variables from .env file


def process_reddit_data():
    load_dotenv('../.env')
    reddit = praw.Reddit(
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("SECRET_KEY"),
        user_agent=os.getenv("REDDIT_GRANT_TYPE"),
        username=os.getenv("REDDIT_USERNAME"),
        password=os.getenv("REDDIT_PASSWORD")
    )

    dfComment = pd.DataFrame(columns=['id', 'timestamp', 'author', 'body', 'title'])
    index_loop = 0

    submission_id = "1b1oevo"
    submission = reddit.submission(submission_id)
    submission.comments.replace_more(limit=None)

    for comment in submission.comments.list():
        dfComment.loc[index_loop, 'id'] = comment.id
        dfComment.loc[index_loop, "timestamp"] = datetime.utcfromtimestamp(comment.created_utc)
        dfComment.loc[index_loop, 'author'] = str(comment.author)
        dfComment.loc[index_loop, 'body'] = comment.body
        dfComment.loc[index_loop, 'title'] = submission.title
        index_loop += 1

    dfComment['dt'] = pd.to_datetime(dfComment['timestamp']).dt.strftime('%Y-%m-%d')
    dfComment['author'] = dfComment['author'].apply(lambda x: str(x))

    data_folder = './notebooks/data/'

    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    ls_dt = sorted(list(dfComment['dt'].unique()))

    print(dfComment)

    return None

if __name__ == "__main__":
    process_reddit_data()

          id            timestamp            author  \
0    kshgkpz  2024-02-28 03:39:02        BroBearhug   
1    kshe4xv  2024-02-28 03:22:30    ferrerobanmian   
2    ksiu1pk  2024-02-28 11:56:24       brocccoli26   
3    ksiwrv7  2024-02-28 12:20:50    LanaWish2BRich   
4    ksivk52  2024-02-28 12:10:16     Great-Cod1685   
..       ...                  ...               ...   
244  ksi8oaw  2024-02-28 07:50:12           jamsloo   
245  ksigiuc  2024-02-28 09:25:05  lurkingeternally   
246  ksio2tq  2024-02-28 10:54:35           jamsloo   
247  ksisq0g  2024-02-28 11:43:46  lurkingeternally   
248  ksiw7le  2024-02-28 12:15:58           jamsloo   

                                                  body  \
0    Almost 5 years with the firm and now it feels ...   
1                                 lunch time jiak simi   
2    big boss is obviously bias and doesn't value m...   
3    Reasons why I am constantly unhappy: poor, sin...   
4    I don’t understand why IMDA would allow Medi

In [64]:
pd.set_option('display.max_colwidth', None)
df2 = pd.read_parquet('/Users/johnnytay/Library/CloudStorage/OneDrive-Personal/My NUS Mtech EBAC course/Semester 3/Practice Module/bead_pyspark/proj_radical_sparks/reddit_data/1b1oevo/1b1oevo_2024-03-02.parquet')
df2

,id,timestamp,author,body,title,dt
0,ksymfqs,2024-03-02 04:48:02,singapore-ModTeam,"Hi LanJiaoDuaKee,\n\nYour submission is behind a paywall, you need to include a summarised but not copied version of it.","US’s Singapore Envoy Is Said to Berate Staff, Waste Money",2024-03-02
1,ksyk0rj,2024-03-02 04:27:12,LanJiaoDuaKee,he was a guest on the daily ketchup podcast \n\n[https://www.youtube.com/watch?v=S6ynOPWi6aE](https://www.youtube.com/watch?v=S6ynOPWi6aE),"US’s Singapore Envoy Is Said to Berate Staff, Waste Money",2024-03-02
2,ksykxoj,2024-03-02 04:35:04,Bitter_Truth_1254,Non paywall link please,"US’s Singapore Envoy Is Said to Berate Staff, Waste Money",2024-03-02
3,ksylva9,2024-03-02 04:43:04,water4ever,May be he is not sure he is in Singapore or China?,"US’s Singapore Envoy Is Said to Berate Staff, Waste Money",2024-03-02
4,ksz6wqv,2024-03-02 08:24:52,Status_Collection383,any relation to the business school,"US’s Singapore Envoy Is Said to Berate Staff, Waste Money",2024-03-02
...,...,...,...,...,...,...
6569,ksym3ie,2024-03-02 04:45:04,gxchung1,The guy look like Steven Lim leh,S'pore streamer Kiaraakitty banned from entering Taiwan for up to 5 years over staged egg-pelting attack,2024-03-02
6922,ksyc9oe,2024-03-02 03:25:09,No_Pension9902,No forbidden scrolls from IMH.,Man who killed ex-fiancee is first person to be executed for murder in Singapore since 2019,2024-03-02
6924,ksydq50,2024-03-02 03:36:24,Noobcakes19,"They tried edo tensei but, failed. \n\nMedical practitioners were puzzled in the use of sign languages by them",Man who killed ex-fiancee is first person to be executed for murder in Singapore since 2019,2024-03-02
6984,ksyrqvn,2024-03-02 05:36:24,ForTheGameGMEr,The dude be like Cumming to this for his ego porn with all the $66k comments,Woman who ‘brought nothing but grief’ to brief marriage with millionaire gets $5k in divorce,2024-03-02


In [ ]:
df1 = pd.read_parquet('/Users/johnnytay/Library/CloudStorage/OneDrive-Personal/My NUS Mtech EBAC course/Semester 3/Practice Module/bead_pyspark/proj_radical_sparks/reddit_data/1b1f5ob/1b1f5ob_2024-02-27.parquet')
df1

In [ ]:
def retrieve_list_of_submission_id():
    reddit = praw.Reddit(
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("SECRET_KEY"),
        user_agent=os.getenv("REDDIT_GRANT_TYPE"),
        username=os.getenv("REDDIT_USERNAME"),
        password=os.getenv("REDDIT_PASSWORD")
    )

    submissions = []

    for submission in reddit.subreddit("Singapore").new(limit=1000):
        submissions.append(submission.id)
        pd.DataFrame(submissions).to_csv('1000_submission_ids.csv')
    
    return submissions

retrieve_list_of_submission_id()

In [85]:
def retrieve_list_of_submission_id(subreddit_name_list):
        reddit = praw.Reddit(
            client_id=os.getenv("CLIENT_ID"),
            client_secret=os.getenv("SECRET_KEY"),
            user_agent=os.getenv("REDDIT_GRANT_TYPE"),
            username=os.getenv("REDDIT_USERNAME"),
            password=os.getenv("REDDIT_PASSWORD")
        )

        submissions = []

        for subreddit_name in subreddit_name_list:
            for submission in reddit.subreddit(subreddit_name).new(limit=100):
                submissions.append(submission.id)
                pd.DataFrame(submissions).to_csv('1000_submission_ids.csv')
            
        return submissions

In [87]:
listing = retrieve_list_of_submission_id(['Singapore', 'SGExams'])
len(listing)

200

In [80]:
def get_submission_ids(file):

        df = pd.read_csv(file)
        submission_ids = df['submission_id'].tolist()
        submission_titles = df['submission_title'].tolist()
        
        return submission_ids[:1], submission_titles[:1]
    

def process_reddit_data(file):
        
        reddit = praw.Reddit(
            client_id=os.getenv("CLIENT_ID"),
            client_secret=os.getenv("SECRET_KEY"),
            user_agent=os.getenv("REDDIT_GRANT_TYPE"),
            username=os.getenv("REDDIT_USERNAME"),
            password=os.getenv("REDDIT_PASSWORD")
        )

        dfComment = pd.DataFrame(columns=['id', 'timestamp', 'author', 'body', 'title'])
        index_loop = 0

        # submission_ids_list, submission_titles = get_submission_ids(file)
        submission_ids_list, submission_titles = (['1b1oevo', '1b1f5ob'], ['/r/singapore random discussion and small questions thread for February 28, 2024', 'Woman to be charged after US$4.2 million in cryptocurrency allegedly misappropriated'])
        # submission_ids_list = '1b1oevo'
        # submission_titles = '/r/singapore random discussion and small questions thread for February 28, 2024'

        for submission_id, submission_title in zip(submission_ids_list, submission_titles):
            submission = reddit.submission(submission_id)
            submission.comments.replace_more(limit=None)

            dfComment = pd.DataFrame(columns=['id', 'timestamp', 'author', 'body', 'title'])
            index_loop = 0

            for comment in submission.comments.list():
                dfComment.loc[index_loop, 'id'] = comment.id
                dfComment.loc[index_loop, "timestamp"] = datetime.utcfromtimestamp(comment.created_utc)
                dfComment.loc[index_loop, 'author'] = str(comment.author)
                dfComment.loc[index_loop, 'body'] = comment.body
                dfComment.loc[index_loop, 'title'] = submission_title
                index_loop += 1

            dfComment['dt'] = pd.to_datetime(dfComment['timestamp']).dt.strftime('%Y-%m-%d')
            dfComment['author'] = dfComment['author'].apply(lambda x: str(x))

            data_folder = f'./proj_radical_sparks/data/{submission_id}/'
            print(data_folder)

            if not os.path.exists(data_folder):
                os.makedirs(data_folder)

            ls_dt = sorted(list(dfComment['dt'].unique()))

            print(dfComment)

            # Process and save the data

            # for dt in ls_dt:
            #     file_name = f'{submission_id}_{dt}.parquet'
            #     dfComment_daily = dfComment[dfComment['dt'] == dt]

            #     arrow_table = pa.Table.from_pandas(dfComment_daily)
            #     file_path = os.path.join(data_folder, file_name)
            #     pq.write_table(arrow_table, file_path)

            #     print(arrow_table)

            # Delete dfComment
            del dfComment

        return None

            # for dt in ls_dt:
            #     file_name = f'{submission_id}_{dt}.parquet'
            #     dfComment_daily = dfComment[dfComment['dt'] == dt]

            #     arrow_table = pa.Table.from_pandas(dfComment_daily)
            #     file_path = os.path.join(data_folder, file_name)
            #     pq.write_table(arrow_table, file_path)

            #     print(arrow_table)

        return None

In [81]:
process_reddit_data('../proj_radical_sparks/new_100_submission.csv')

./proj_radical_sparks/data/1b1oevo/
          id            timestamp            author  \
0    kshgkpz  2024-02-28 03:39:02        BroBearhug   
1    kshe4xv  2024-02-28 03:22:30    ferrerobanmian   
2    ksiu1pk  2024-02-28 11:56:24       brocccoli26   
3    ksiwrv7  2024-02-28 12:20:50    LanaWish2BRich   
4    ksivk52  2024-02-28 12:10:16     Great-Cod1685   
..       ...                  ...               ...   
244  ksi8oaw  2024-02-28 07:50:12           jamsloo   
245  ksigiuc  2024-02-28 09:25:05  lurkingeternally   
246  ksio2tq  2024-02-28 10:54:35           jamsloo   
247  ksisq0g  2024-02-28 11:43:46  lurkingeternally   
248  ksiw7le  2024-02-28 12:15:58           jamsloo   

                                                                                                                                                                                                                                                                                                               

In [77]:
process_reddit_data('../proj_radical_sparks/new_100_submission.csv')

./proj_radical_sparks/data/1b1oevo/
pyarrow.Table
id: string
timestamp: timestamp[us]
author: string
body: string
title: string
dt: string
__index_level_0__: int64
----
id: [["ksfxfx7","ksg2n3l","ksgcsbd","ksgg14w","ksgbvk8"]]
timestamp: [[2024-02-27 22:00:31.000000,2024-02-27 22:29:34.000000,2024-02-27 23:29:49.000000,2024-02-27 23:49:50.000000,2024-02-27 23:24:11.000000]]
author: [["blackwoodsix","Traxgen","jamsloo","silentscope90210","JokerMother"]]
body: [["Good morning all have a good Wednesday and stay strong, stay safe and stay healthy!! Three more days to weekend jiayou!!","First time downloading dating apps in more than a year, and my first match in more than a week is a catfish 🥲

Sian dating is so hard","I just want a way to play wordle with as many people strangers whoever without any ads !!","Damn... Friend's mum passed on. That's already a few deaths I know of this past few months.","3 more days to taylor swift opening night!"]]
title: [["/r/singapore random discussion an

In [82]:
import glob
import pyarrow.parquet as pq
import os
import pandas as pd

# FILEPATH: /Users/johnnytay/Library/CloudStorage/OneDrive-Personal/My NUS Mtech EBAC course/Semester 3/Practice Module/bead_pyspark/notebooks/migrated_from_AWS.ipynb

folder_path = '../proj_radical_sparks/reddit_data_folder/'

# Get a list of all Parquet files in the folder and its subfolders
parquet_files = glob.glob(os.path.join(folder_path, '**/*.parquet'), recursive=True)

# Read each Parquet file and append the data to a list
data = []
for file in parquet_files:
    table = pq.read_table(file)
    df = table.to_pandas()
    data.append(df)

# Concatenate all the data into a single DataFrame
df_combined = pd.concat(data, ignore_index=True)

df_combined


,id,timestamp,author,body,title,dt
0,kss1s65,2024-03-01 00:18:44,EastBeasteats,"Make meta contribute to a scam victims compensation fund. \n\n\nThe principle is simple - he who profits, bears the cost.",Sun Xueling calls out Meta for not working with MHA to fight e-commerce scams on its platforms,2024-03-01
1,ksu70tb,2024-03-01 11:42:41,Gold_Retirement,"I think she would make better progress if she takes real action, rather than monitoring the situation and then bitch about it when the situation doesn't improve.",Sun Xueling calls out Meta for not working with MHA to fight e-commerce scams on its platforms,2024-03-01
2,kss0bqd,2024-03-01 00:09:23,TadpoleNational2222,Block FB and WhatsApp in Singapore. Heavily fine them if needed to. Let other platforms replace them.,Sun Xueling calls out Meta for not working with MHA to fight e-commerce scams on its platforms,2024-03-01
3,kss3760,2024-03-01 00:27:41,jupiter1_,I posted this few months back to regulate \n\nSome dude was arguing with me why should Meta bother. It's all about ethics man.,Sun Xueling calls out Meta for not working with MHA to fight e-commerce scams on its platforms,2024-03-01
4,kssihid,2024-03-01 02:06:23,Anderweise,"Meta has their regional HQ in spore? Does the govt dare to do anything, or just empty words only? I’m sure we have enough tools to compel or punish them already. Waiting for what?",Sun Xueling calls out Meta for not working with MHA to fight e-commerce scams on its platforms,2024-03-01
...,...,...,...,...,...,...
7860,ksfxfx7,2024-02-27 22:00:31,blackwoodsix,"Good morning all have a good Wednesday and stay strong, stay safe and stay healthy!! Three more days to weekend jiayou!!","/r/singapore random discussion and small questions thread for February 28, 2024",2024-02-27
7861,ksg2n3l,2024-02-27 22:29:34,Traxgen,"First time downloading dating apps in more than a year, and my first match in more than a week is a catfish 🥲\n\nSian dating is so hard","/r/singapore random discussion and small questions thread for February 28, 2024",2024-02-27
7862,ksgcsbd,2024-02-27 23:29:49,jamsloo,I just want a way to play wordle with as many people strangers whoever without any ads !!,"/r/singapore random discussion and small questions thread for February 28, 2024",2024-02-27
7863,ksgg14w,2024-02-27 23:49:50,silentscope90210,Damn... Friend's mum passed on. That's already a few deaths I know of this past few months.,"/r/singapore random discussion and small questions thread for February 28, 2024",2024-02-27


In [83]:
df_combined.duplicated().sum()

0

In [69]:
def get_submission_ids(file):

        df = pd.read_csv(file)
        submission_ids = df['submission_id'].tolist()
        submission_titles = df['submission_title'].tolist()
        
        return submission_ids[:1], submission_titles[:1]

get_submission_ids('../proj_radical_sparks/new_100_submission.csv')

(['1b4ff0t'], ['US’s Singapore Envoy Is Said to Berate Staff, Waste Money'])

In [44]:
submission_ids_list, submission_titles = get_submission_ids('../proj_radical_sparks/new_100_submission.csv')

In [45]:
for submission_id, submission_title in zip(submission_ids_list, submission_titles):
    print(submission_title)

US’s Singapore Envoy Is Said to Berate Staff, Waste Money
What happened to the smartest kid/top scorer from your school?
Commentary: Uproar over lantern festival and drone show cancellations highlight importance of good PR
The Big Read: Malaysia's brain drain to Singapore — winners and losers in the long run not so clear-cut
BBC: Bad blood over Singapore Taylor Swift tour subsidies
Dignity Kitchen hopes to raise $200,000 for restoration works after fire
/r/singapore random discussion and small questions thread for March 02, 2024
Singapore aims to build AI talent pool through accelerated masters programme, visiting professorships
Forum: Naming the pre-schools allegedly involved in child abuse cases could lead to more accountability
Decathlon has stopped their lifetime refund and exchange policy for members, now limited to 365 days
